## Abstract

This notebook formulates the Optimal Power Flow (OPF) problem for electrical power systems as a Quadratic Unconstrained Binary Optimization (QUBO) model suitable for quantum annealing and gate-based quantum optimization algorithms.

OPF is a foundational problem in energy systems engineering, governing the efficient and stable dispatch of generation subject to network, physical, and operational constraints. By expressing OPF in QUBO form, this work enables direct application of quantum optimization techniques to large-scale energy infrastructure problems, establishing a bridge between power systems engineering and quantum computation.


## Why Quantum Optimization for Power Systems

Modern electrical grids are becoming increasingly complex due to renewable integration, distributed generation, and real-time operational constraints. Classical OPF solvers must repeatedly solve large nonconvex optimization problems under tight time requirements, often relying on approximations that trade optimality for speed.

Quantum optimization methods offer a fundamentally different approach by exploiting quantum parallelism to explore complex solution landscapes. Reformulating OPF as a QUBO problem allows it to be mapped directly to quantum annealers and hybrid quantum–classical optimizers, providing a potential pathway toward faster or more scalable grid optimization in future quantum-enabled control systems.


## From Optimal Power Flow to QUBO

The classical Optimal Power Flow problem is typically formulated as a constrained nonlinear optimization task minimizing generation cost subject to power balance, line flow, and generator limits.

To enable quantum optimization, the constrained OPF formulation is transformed into an unconstrained quadratic objective using penalty methods. Continuous variables are discretized into binary encodings, and all constraints are absorbed into the objective function. The resulting Quadratic Unconstrained Binary Optimization (QUBO) form is compatible with quantum annealing and variational quantum algorithms.

This transformation enables exploration of quantum-native optimization strategies for energy system control.


### Constraint Handling Note

In this notebook, feasibility is enforced primarily through the binary-to-real decoding map, which restricts decoded generation and voltage values to physically valid operating ranges. As a result, explicit penalty terms for box constraints are unnecessary in this formulation. This implementation therefore represents a proof-of-principle QUBO-style discretization and optimization workflow for an OPF-inspired objective. Extension to full OPF formulations with network and power-balance constraints is a natural next step and is outlined in the Future Directions section.


## **Power Flow Optimization Problem**

The Optimal Power Flow (OPF) problem plays a critical role in the planning and operation of modern power systems. As power grids continue to grow in scale and complexity (driven by the emergence of smart grid technologies, widespread integration of renewable energy sources, and the deployment of energy storage systems) the interest in OPF has significantly increased.

These developments introduce new challenges, particularly due to the uncertainty and variability associated with renewables and storage. In this evolving landscape, OPF has become an essential tool for resource allocation, improving network efficiency, and ensuring reliable system operation under uncertain conditions.


A simplified Power Flow Optimization Framework is given as follows: \
 \
1.*Fuel Cost Minimization Fuction*: $f_1 = ∑_i (a_iP^2_i + b_iP_i)$

2.*Power Loss Minimization*:$f_2 = ∑_i 2g_iV^2_i-0.95V^2_i $ \
\
where $P_i$ is the Active Power Generation at bus $i$ and $V_i$ is the Voltage magnitude at bus $i$.
\
\
$Min → w_1f_1 + w_2f_2$ (wieighted sum approach)\
\
subject to \
\
$P_i \in [P^{min}_i, P^{max}_i]$
$V_i \in [V^{min}_i, V^{max}_i]$ \
\
where $i \in [1,3]$.

*Fuel Cost Coefficients*: \
$a_1=0.01,a_2=0.02,a_3=0.03$ \
$b_1=0.02,b_2=0.02,b_3=0.01$ \
 \
*Power Loss Coefficients*: \
$g_1=0.1,g_2=0.2,g_3=0.3$


\
*Power Output and Voltage Constraints*: \
$P_i \in [0, 100]$  MW and \
$V_i \in [0.95, 1.05]$ for all $i \in [1,3]$\
\
Consider equal weight on both objective functions ⇒ $(w_1,w_2)=(0.5,0.5)$

Classical base line

In [1]:
import numpy as np
from scipy.optimize import minimize

# --- Power Flow Optimization Problem Definition ---

# Coefficients for Fuel Cost Minimization (f1)
a = np.array([0.01, 0.02, 0.03])
b = np.array([0.02, 0.02, 0.01])

# Coefficients for Power Loss Minimization (f2)
g = np.array([0.1, 0.2, 0.3])

# Weights for the combined objective function
w1 = 0.5
w2 = 0.5

# Define the objective function
# x will be a 6-element array: [P1, P2, P3, V1, V2, V3]
def power_flow_objective(x):
    P = x[:3]  # Active Power Generation for buses 1, 2, 3
    V = x[3:]  # Voltage magnitude for buses 1, 2, 3

    # Fuel Cost Minimization Fuction: f1 = sum(a_i*P_i^2 + b_i*P_i)
    f1 = np.sum(a * P**2 + b * P)

    # Power Loss Minimization: f2 = sum(2*g_i*V_i^2 - 0.95*V_i^2)
    f2 = np.sum((2 * g - 0.95) * V**2)

    # Combined objective: w1*f1 + w2*f2
    return w1 * f1 + w2 * f2

# --- Constraints ---
# Power Output and Voltage Constraints:
# P_i in [10, 100] MW
# V_i in [0.95, 1.05]

# Bounds for P (3 variables) and V (3 variables)
P_bounds = [(10, 100) for _ in range(3)]
V_bounds = [(0.95, 1.05) for _ in range(3)]

# Combine all bounds: [ (P1_min, P1_max), ..., (V3_min, V3_max) ]
bounds = P_bounds + V_bounds

# Initial guess for P and V values
# Start with values within the allowed range
x0 = np.array([50.0, 50.0, 50.0, 1.0, 1.0, 1.0])

# --- Run the optimization ---
# SLSQP (Sequential Least Squares Programming) is suitable for constrained nonlinear optimization
result = minimize(power_flow_objective, x0, bounds=bounds, method='SLSQP')

# --- Print results ---
if result.success:
    optimal_P = result.x[:3]
    optimal_V = result.x[3:]
    print("Optimal solution found:")
    print(f"Optimal Active Power (P): {optimal_P}")
    print(f"Optimal Voltage (V): {optimal_V}")
    print(f"Minimum Objective Function Value = {result.fun:.6f}")
else:
    print("Optimization failed:")
    print(result.message)

Optimal solution found:
Optimal Active Power (P): [10. 10. 10.]
Optimal Voltage (V): [1.05 1.05 1.05]
Minimum Objective Function Value = 2.340438


## QUBO-Style Optimization via Simulated Annealing

The binary-encoded OPF objective defines a QUBO-style optimization problem over a discrete bitstring space. While quantum annealers and variational quantum algorithms are natural solvers for this class of problems, simulated annealing is used here as a classical stand-in optimizer to validate the formulation and benchmark solution quality.

This allows direct assessment of the QUBO mapping independent of current quantum hardware limitations, while preserving compatibility with future quantum optimization backends.



In [2]:
import numpy as np
import itertools

# --- QUBO Parameters for Power Flow Optimization ---

# Discretization parameters for P and V variables
num_bits_P = 10 # Number of bits to represent each P_i variable
num_bits_V = 10 # Number of bits to represent each V_i variable

# Penalty coefficient for converting constrained problem to unconstrained QUBO
# This will be used if we need to penalize constraint violations.
lambda_penalty = 1000

# The actual bounds for P and V are defined in the problem description:
# P_i in [10, 100] MW
# V_i in [0.95, 1.05]
# These bounds will be used when mapping binary variables to physical values.

In [3]:
# Bounds for P and V
P_min, P_max = 10, 100
V_min, V_max = 0.95, 1.05

# Scaling step sizes for P and V
levels_P = 2 ** num_bits_P
step_P = (P_max - P_min) / (levels_P - 1)

levels_V = 2 ** num_bits_V
step_V = (V_max - V_min) / (levels_V - 1)

# Helper function to convert a binary array for a single variable to its scalar real value
def bits_to_scalar(bits_array, lower_bound, step_size):
    # Assuming bits_array[0] is LSB, bits_array[num_bits-1] is MSB
    integer_value = sum(bit * (2 ** i) for i, bit in enumerate(bits_array))
    return lower_bound + integer_value * step_size

# Function to map a complete binary vector to all 6 real values (P1, P2, P3, V1, V2, V3)
def binary_vector_to_real_values(binary_vector):
    real_values = []
    current_bit_idx = 0

    # Process P variables (3 of them)
    for _ in range(3):
        p_bits = binary_vector[current_bit_idx : current_bit_idx + num_bits_P]
        p_val = bits_to_scalar(p_bits, P_min, step_P)
        real_values.append(p_val)
        current_bit_idx += num_bits_P

    # Process V variables (3 of them)
    for _ in range(3):
        v_bits = binary_vector[current_bit_idx : current_bit_idx + num_bits_V]
        v_val = bits_to_scalar(v_bits, V_min, step_V)
        real_values.append(v_val)
        current_bit_idx += num_bits_V

    return np.array(real_values)

# Total number of binary variables for the entire problem
total_num_binary_variables = (3 * num_bits_P) + (3 * num_bits_V)

In [4]:
# Objective function for the QUBO framework, refitted with penalty structure
def objective_with_penalty(binary_vector):
    # Convert binary vector to real P and V values
    real_values = binary_vector_to_real_values(binary_vector)
    P = real_values[:3]
    V = real_values[3:]

    # Coefficients for Fuel Cost Minimization (f1) - already defined globally
    # a = np.array([0.01, 0.02, 0.03])
    # b = np.array([0.02, 0.02, 0.01])

    # Coefficients for Power Loss Minimization (f2) - already defined globally
    # g = np.array([0.1, 0.2, 0.3])

    # Weights for the combined objective function - already defined globally
    # w1 = 0.5
    # w2 = 0.5

    # Calculate the primary objective value
    f1 = np.sum(a * P**2 + b * P)
    f2 = np.sum((2 * g - 0.95) * V**2)
    primary_objective_value = w1 * f1 + w2 * f2

    # Placeholder for penalty term
    # As the binary_to_real mapping inherently respects the discretization within bounds,
    # no explicit penalty for P and V bounds is needed here. If other complex constraints
    # were present (e.g., total power balance, line limits), they would be added here
    # as penalty_term = max(0, constraint_violation)**2 or similar.
    penalty_term = 0

    return primary_objective_value + lambda_penalty * penalty_term

In [5]:
import random
import math

# --- Simulated Annealing for QUBO --- (Approximate Solver)

def simulated_annealing(objective_func, total_num_binary_variables, max_iterations, initial_temperature, cooling_rate):
    # Initialize a random binary solution
    current_solution = [random.randint(0, 1) for _ in range(total_num_binary_variables)]
    current_energy = objective_func(current_solution)

    best_solution = list(current_solution)
    best_energy = current_energy

    temperature = initial_temperature

    print("Starting Simulated Annealing...")
    print(f"Initial Energy: {current_energy:.6f}")

    for i in range(max_iterations):
        # Create a neighboring solution by flipping a random bit
        neighbor_solution = list(current_solution)
        bit_to_flip = random.randint(0, total_num_binary_variables - 1)
        neighbor_solution[bit_to_flip] = 1 - neighbor_solution[bit_to_flip]

        neighbor_energy = objective_func(neighbor_solution)

        # Acceptance probability
        if neighbor_energy < current_energy or random.random() < math.exp((current_energy - neighbor_energy) / temperature):
            current_solution = list(neighbor_solution)
            current_energy = neighbor_energy

        # Update best solution found so far
        if current_energy < best_energy:
            best_solution = list(current_solution)
            best_energy = current_energy

        # Cool down the temperature
        temperature *= cooling_rate

        if i % (max_iterations // 10) == 0: # Print progress every 10%
            print(f"Iteration {i}/{max_iterations}, Temp: {temperature:.4f}, Current Energy: {current_energy:.6f}, Best Energy: {best_energy:.6f}")

    print("\nSimulated Annealing Finished.")
    return best_solution, best_energy

# --- Run Simulated Annealing ---

# Simulated Annealing parameters (can be tuned)
MAX_ITERATIONS = 500000 # Number of iterations
INITIAL_TEMPERATURE = 100.0 # Starting temperature
COOLING_RATE = 0.999 # Rate at which temperature decreases

best_binary_solution, min_objective_sa = simulated_annealing(
    objective_with_penalty,
    total_num_binary_variables,
    MAX_ITERATIONS,
    INITIAL_TEMPERATURE,
    COOLING_RATE
)

# Decode the best binary solution to real values
optimal_P_qubo = binary_vector_to_real_values(best_binary_solution)[:3]
optimal_V_qubo = binary_vector_to_real_values(best_binary_solution)[3:]

print("\n--- QUBO (Simulated Annealing) Results ---")
print(f"Optimal Active Power (P) from QUBO: {optimal_P_qubo}")
print(f"Optimal Voltage (V) from QUBO: {optimal_V_qubo}")
print(f"Minimum Objective Function Value (QUBO) = {min_objective_sa:.6f}")

Starting Simulated Annealing...
Initial Energy: 277.171597
Iteration 0/500000, Temp: 99.9000, Current Energy: 279.180672, Best Energy: 277.171597
Iteration 50000/500000, Temp: 0.0000, Current Energy: 2.340438, Best Energy: 2.340438
Iteration 100000/500000, Temp: 0.0000, Current Energy: 2.340438, Best Energy: 2.340438
Iteration 150000/500000, Temp: 0.0000, Current Energy: 2.340438, Best Energy: 2.340438
Iteration 200000/500000, Temp: 0.0000, Current Energy: 2.340438, Best Energy: 2.340438
Iteration 250000/500000, Temp: 0.0000, Current Energy: 2.340438, Best Energy: 2.340438
Iteration 300000/500000, Temp: 0.0000, Current Energy: 2.340438, Best Energy: 2.340438
Iteration 350000/500000, Temp: 0.0000, Current Energy: 2.340438, Best Energy: 2.340438
Iteration 400000/500000, Temp: 0.0000, Current Energy: 2.340438, Best Energy: 2.340438
Iteration 450000/500000, Temp: 0.0000, Current Energy: 2.340438, Best Energy: 2.340438

Simulated Annealing Finished.

--- QUBO (Simulated Annealing) Results -

## Discussion

This study demonstrates the feasibility of encoding a simplified Optimal Power Flow problem into QUBO form, enabling its solution using quantum optimization paradigms. While classical solvers remain highly efficient for many OPF instances, their performance can degrade as system size and constraint complexity increase.

The QUBO formulation provides a direct interface between power system optimization and quantum hardware, offering a foundation for future hybrid quantum–classical grid control systems. Importantly, this work focuses on structural feasibility rather than near-term performance superiority, emphasizing methodological integration rather than immediate quantum advantage.


## Limitations

This implementation considers a simplified OPF formulation and small-scale system instances suitable for notebook-based experimentation. Full-scale transmission networks involve thousands of buses and constraints, which remain beyond current quantum hardware capacity.

Additionally, QUBO formulations can introduce large penalty terms that distort the energy landscape, potentially affecting solution quality. As such, results should be interpreted as proof-of-principle demonstrations rather than production-ready optimization pipelines.


### Quantum State Space and Continuous Evolution

A defining distinction between classical and quantum optimization lies in the structure of the underlying state space. Classical algorithms explore configurations sequentially within a discrete or continuous parameter space. Quantum systems, by contrast, evolve within exponentially large Hilbert spaces, where superposition and entanglement allow simultaneous representation of vast numbers of candidate solutions.

In a quantum system of (n) qubits, the computational state resides in a (2^n)-dimensional Hilbert space, enabling quantum algorithms to process global features of the optimization landscape that are inaccessible to purely classical methods. Superposition permits parallel exploration of many configurations, while entanglement encodes high-order correlations between variables, reshaping the effective geometry of the search space.

This capability is particularly relevant for complex engineering systems such as power grids, where objective landscapes are highly nonlinear and strongly coupled. Quantum evolution naturally operates through continuous transformations of quantum states, offering a fundamentally different mode of computation compared to discrete classical search trajectories.

From this perspective, discretized QUBO mappings represent a near-term compromise rather than the ultimate form of quantum optimization. As quantum hardware matures, algorithms that exploit continuous quantum dynamics and high-dimensional entangled state spaces may enable more direct and efficient exploration of complex optimization landscapes, providing a deeper pathway toward quantum advantage in large-scale infrastructure systems.


## Future Directions

Future work will focus on extending the QUBO formulation to larger grid models, incorporating stochastic renewable generation and time-coupled constraints. Integration with hybrid solvers combining classical preprocessing and quantum subproblem optimization represents a promising near-term pathway.

As quantum hardware scales, QUBO-based OPF formulations may support real-time grid control applications, enabling adaptive energy dispatch in increasingly complex power systems.


**References & Study Resources**

1. QUBO: https://en.wikipedia.org/wiki/Quadratic_unconstrained_binary_optimization
2. QUBO: Boros, E., Hammer, P.L. and Tavares, G., 2007. Local search heuristics for quadratic unconstrained binary optimization (QUBO). Journal of Heuristics, 13, pp.99-132.https://dl.acm.org/doi/10.1007/s10732-007-9009-3#core-cited-by
3. Quadratic Programming :Boggs, P.T. and Tolle, J.W., 1995. Sequential quadratic programming. Acta numerica, 4, pp.1-51.
4. Quadratic Programming: https://en.wikipedia.org/wiki/Sequential_quadratic_programming
5. Nonlinear Programming: https://en.wikipedia.org/wiki/Nonlinear_programming
6. Constraint Handling: https://www.sciencedirect.com/science/article/abs/pii/B9780124167438000130
7. Penalty Method: https://en.wikipedia.org/wiki/Penalty_method
8. Quantum Annealing: Cohen, E. and Tamir, B., 2014. D-Wave and predecessors: From simulated to quantum annealing. International Journal of Quantum Information, 12(03), p.1430002.
9. Quantum Annealing: https://en.wikipedia.org/wiki/Quantum_annealing
10. Adiabatic Quantum Computation: https://en.wikipedia.org/wiki/Adiabatic_quantum_computation
11. Quantum Annealing:https://docs.dwavequantum.com/en/latest/quantum_research/quantum_annealing_intro.html
12. Babiker, A., Ahmad, S.S., Ahmed, I., Khalid, M., Abido, M.A. and Al-Ismail, F.S., 2025. Optimal Power Flow: A Review of State-of-the-Art Techniques and Future Perspectives. IEEE Access.